# MATH7016 Deep Learning
### Coursework 2025

In [12]:
import torch
from torch import Tensor
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import numpy as np
from numpy.random import seed
import tensorflow as tf
from tensorflow import keras
from keras import backend as K

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects
from sklearn.preprocessing import MinMaxScaler

tf.random.set_seed(42)

In [34]:
train_df = pd.read_csv('DL-2025-CW-data/Data_A.csv')
test_df = pd.read_csv('DL-2025-CW-data/Data_B_nolabels.csv')

columns_names = ['target']
level_cols=[]
for i in range(4):
    level_cols +=[f'ask_price{i+1}',f'ask_size{i+1}',f'bid_price{i+1}',f'bid_size{i+1}']
change_cols =  []
for i in range(5):
    change_cols+=[f'midprice_change{i+1}']
columns_names+=level_cols+change_cols
train_df.columns = columns_names
test_df.columns = columns_names[1:]
test_df.head()

,ask_price1,ask_size1,bid_price1,bid_size1,ask_price2,ask_size2,bid_price2,bid_size2,ask_price3,ask_size3,...,bid_size3,ask_price4,ask_size4,bid_price4,bid_size4,midprice_change1,midprice_change2,midprice_change3,midprice_change4,midprice_change5
0,696400.0,16,696000.0,12,696500.0,57,695900.0,118,696600.0,100,...,262,696700.0,150,695700.0,104,1,0,1,0,1
1,740800.0,2,740400.0,20,741000.0,60,740200.0,27,741200.0,156,...,31,741300.0,200,740000.0,170,0,1,0,0,1
2,730900.0,1,730200.0,230,731000.0,111,730100.0,86,731100.0,42,...,136,731200.0,100,729900.0,132,1,1,0,0,1
3,630600.0,100,630300.0,69,630700.0,110,630200.0,2,630800.0,219,...,1,630900.0,101,630000.0,104,0,1,0,0,0
4,851100.0,579,850300.0,25,851200.0,17,850100.0,287,851400.0,307,...,270,851500.0,223,849900.0,72,1,0,1,0,1


In [35]:
train_df

,target,ask_price1,ask_size1,bid_price1,bid_size1,ask_price2,ask_size2,bid_price2,bid_size2,ask_price3,...,bid_size3,ask_price4,ask_size4,bid_price4,bid_size4,midprice_change1,midprice_change2,midprice_change3,midprice_change4,midprice_change5
0,0,650400.0,501,650200.0,106,650500.0,245,650100.0,259,650600.0,...,328,650700.0,141,649900.0,277,0,0,1,1,1
1,1,636200.0,153,635800.0,150,636300.0,100,635700.0,15,636400.0,...,110,636500.0,105,635500.0,201,0,1,0,1,0
2,0,724800.0,4,724500.0,14,724900.0,50,724300.0,312,725100.0,...,100,725200.0,379,724100.0,56,1,0,0,1,1
3,0,622900.0,110,622700.0,100,623000.0,523,622600.0,300,623100.0,...,249,623200.0,605,622400.0,200,1,0,1,1,0
4,1,620100.0,374,619900.0,110,620200.0,495,619800.0,210,620300.0,...,601,620400.0,310,619600.0,100,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,0,429000.0,400,428800.0,500,429100.0,300,428700.0,700,429200.0,...,655,429300.0,300,428500.0,400,1,0,1,1,1
199995,1,413600.0,100,413500.0,500,413700.0,611,413400.0,457,413800.0,...,400,413900.0,300,413200.0,800,0,1,0,1,0
199996,0,381900.0,401,381800.0,100,382000.0,705,381700.0,800,382100.0,...,400,382200.0,400,381500.0,400,1,0,1,0,0
199997,1,443000.0,100,442900.0,29,443100.0,300,442800.0,100,443200.0,...,400,443300.0,800,442600.0,200,0,1,0,1,0


In [15]:
def calc_wap(df, bid_p, ask_p, bid_s, ask_s):
    return (df[bid_p] * df[ask_s] + df[ask_p] * df[bid_s]) / (df[bid_s] + df[ask_s])

def calc_wmp(df, bid_p, ask_p, bid_s, ask_s):
    return (df[bid_p] * df[bid_s] + df[ask_p] * df[ask_s]) / (df[bid_s] + df[ask_s])

def mid_price(df,  bid_p, ask_p):
    return (df[bid_p] + df[ask_p])/2




In [133]:
def preprocessor(df):
    
    # Step 1: Calculate price & volume features
    for i in range(1,5):
        df[f"wap{i}"] = calc_wap(df, f"bid_price{i}", f"ask_price{i}", f"bid_size{i}", f"ask_size{i}")
        df[f"wmp{i}"] = calc_wmp(df, f"bid_price{i}", f"ask_price{i}", f"bid_size{i}", f"ask_size{i}")
        df[f"price_spread{i}"] = (df[f"ask_price{i}"] - df[f"bid_price{i}"]) / ((df[f"ask_price{i}"] + df[f"bid_price{i}"]) / 2)
        df[f'midprice{i}']=mid_price(df,f'bid_price{i}',f'ask_price{i}')
        df[f"bid_size_diff{i}"] = (df[f"bid_size{i}"] / df[f"bid_size{i}"]) - 1
        df[f"ask_size_diff{i}"] = (df[f"ask_size{i}"] / df[f"ask_size{i}"]) - 1
        df[f"bid_spread{i}"] = df[f"bid_price{i}"] - df[f"bid_price{i}"]
        df[f"ask_spread{i}"] = df[f"ask_price{i}"] - df[f"ask_price{i}"]
        df[f"order_imbalance{i}"] = (df[f"bid_size{i}"] - df[f"ask_size{i}"]) / (df[f"bid_size{i}"] + df[f"ask_size{i}"])
        df[f'bid_ask_ratio{i}'] = df[f'bid_size{i}'] / (df[f'bid_size{i}'] + df[f'ask_size{i}'])
        
    # Price features
    df["wap_balance"] = (df["wap1"] - df["wap2"]).abs()
    df['bid_depth_ratio'] = df['bid_size1'] / (df[['bid_size1','bid_size2','bid_size3','bid_size4']].sum(axis=1))
    df['ask_depth_ratio'] = df['ask_size1'] / (df[['ask_size1','ask_size2','ask_size3','ask_size4']].sum(axis=1))


    # Volume features
    df["total_volume"] = df[["ask_size1", "ask_size2", "bid_size1", "bid_size2"]].sum(axis=1)
    df["volume_imbalance"] = (
        (df["ask_size1"] + df["ask_size2"]) - (df["bid_size1"] + df["bid_size2"])
    ).abs()


    # Order imbalances
    
    df["order_imbalance_total"] = (
        (df["bid_size1"] + df["bid_size2"] - df["ask_size1"] - df["ask_size2"]) /
        (df["bid_size1"] + df["bid_size2"] + df["ask_size1"] + df["ask_size2"])
    )


    # Step 3: Drop raw order book columns
    df = df.drop(columns=level_cols)
    
    return df


In [134]:
train_nn = preprocessor(train_df)
test_nn = preprocessor(test_df)

In [135]:
train_nn

,target,midprice_change1,midprice_change2,midprice_change3,midprice_change4,midprice_change5,wap1,wmp1,price_spread1,midprice1,...,bid_pressure4,ask_pressure4,pressure_diff,spread_pct1,spread_pct2,spread_pct3,spread_pct4,wap_wmp_diff,bid_depth_ratio,ask_depth_ratio
0,0,0,0,1,1,1,650234.925865,650365.074135,0.000308,650300.0,...,180022300.0,91748700.0,-42741000.0,0.000308,0.000615,0.000923,0.001230,368.140679,0.109278,0.484058
1,1,0,1,0,1,0,635998.019802,636001.980198,0.000629,636000.0,...,127735500.0,66832500.0,-20704100.0,0.000629,0.000943,0.001258,0.001572,-256.790090,0.315126,0.301181
2,0,1,0,0,1,1,724733.333333,724566.666667,0.000414,724650.0,...,40549600.0,274850800.0,-73665800.0,0.000414,0.000828,0.001242,0.001518,-395.860799,0.029046,0.006861
3,0,1,0,1,1,0,622795.238095,622804.761905,0.000321,622800.0,...,124480000.0,377036000.0,-515769300.0,0.000321,0.000642,0.000963,0.001285,-685.457745,0.117786,0.065632
4,1,0,1,0,1,0,619945.454545,620054.545455,0.000323,620000.0,...,61960000.0,192324000.0,-228756700.0,0.000323,0.000645,0.000968,0.001290,-391.276631,0.107738,0.269258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,0,1,0,1,1,1,428911.111111,428888.888889,0.000466,428900.0,...,171400000.0,128790000.0,151223000.0,0.000466,0.000933,0.001399,0.001865,201.974174,0.221729,0.210526
199995,1,0,1,0,1,0,413583.333333,413516.666667,0.000242,413550.0,...,330560000.0,124170000.0,307733100.0,0.000242,0.000725,0.001209,0.001693,341.590058,0.231803,0.070872
199996,0,1,0,1,0,0,381819.960080,381880.039920,0.000262,381850.0,...,152600000.0,152880000.0,-203192300.0,0.000262,0.000786,0.001309,0.001833,-185.271077,0.058824,0.179821
199997,1,0,1,0,1,0,442922.480620,442977.519380,0.000226,442950.0,...,88520000.0,354640000.0,-342105900.0,0.000226,0.000677,0.001129,0.001580,-553.610188,0.039781,0.066667


In [138]:

def swish(x, beta = 1):
    return (x * K.sigmoid(beta * x))


get_custom_objects().update({'swish': Activation(swish)})

hidden_units = (128,64)
feature_cols = [c for c in train_nn.columns if c != 'target']
num_cols = [c for c in train_nn.columns if c != 'target' and c not in change_cols]
num_features = len(num_cols)


def base_model():

    change_input = keras.Input(shape=(5,), name='change_data')
    num_input = keras.Input(shape=(num_features,), name='num_data')

    change_dense = keras.layers.Dense(16, activation='swish')(change_input)

    x = keras.layers.Concatenate()([change_dense, num_input])

    for n_hidden in hidden_units:
        x = keras.layers.Dense(n_hidden, activation='swish')(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dropout(0.2)(x)
    out = keras.layers.Dense(1, activation='sigmoid', name='prediction')(x)

    model = keras.Model(
        inputs=[change_input, num_input],
        outputs=out
    )

    return model





In [139]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

train_nn = train_nn.fillna(0)
test_nn = test_nn.fillna(0)

X = train_nn[feature_cols]
y = train_nn['target']

test_predictions_nn = np.zeros(test_nn.shape[0])
scores_folds = {'NN_model': []}

# KFold 
kfolds = 5
kf = KFold(n_splits=kfolds, shuffle=True, random_state=42)

counter = 1

for train_index, val_index in kf.split(X):

    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_val = X.iloc[val_index]
    y_val = y.iloc[val_index]
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    X_train_change = X_train_scaled[:, [feature_cols.index(c) for c in change_cols]]
    X_train_num = X_train_scaled[:, [feature_cols.index(c) for c in num_cols]]
    X_val_change = X_val_scaled[:, [feature_cols.index(c) for c in change_cols]]
    X_val_num = X_val_scaled[:, [feature_cols.index(c) for c in num_cols]]


    model = base_model()
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.005),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # ---- callbacks ----
    # Early stopping callback
    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=20, verbose=0,
        mode='min', restore_best_weights=True)

    # Learning rate reduction callback 
    plateau = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=7, verbose=0,
        mode='min')
    model.fit(
        [X_train_change, X_train_num],
        y_train,
        batch_size=1024,
        epochs=200,
        validation_data=([X_val_change, X_val_num], y_val),
        callbacks=[es, plateau],
        shuffle=True,
        verbose=1
    )
    
    preds = model.predict([X_val_change, X_val_num]).reshape(-1)
    score = np.mean((preds.round() != y_val.values).astype(float)) 
    print(f'Fold {counter}: Accuracy = {1 - score:.5f}')
    scores_folds['NN_model'].append(1 - score)
    
    X_test = test_nn[feature_cols]
    X_test_scaled = scaler.transform(X_test)

    X_test_change = X_test_scaled[:, [feature_cols.index(c) for c in change_cols]]
    X_test_num = X_test_scaled[:, [feature_cols.index(c) for c in num_cols]]


    test_predictions_nn += model.predict([X_test_change, X_test_num]).reshape(-1) / kfolds
    
    counter += 1


print("CV folds scores:", scores_folds['NN_model'])

Epoch 1/200
157/157 [==============================] - 2s 8ms/step - loss: 0.6040 - accuracy: 0.6799 - val_loss: 0.5917 - val_accuracy: 0.6858 - lr: 0.0050
Epoch 2/200
157/157 [==============================] - 1s 6ms/step - loss: 0.5743 - accuracy: 0.6980 - val_loss: 0.6890 - val_accuracy: 0.6863 - lr: 0.0050
Epoch 3/200
157/157 [==============================] - 1s 8ms/step - loss: 0.5671 - accuracy: 0.7005 - val_loss: 0.8459 - val_accuracy: 0.6862 - lr: 0.0050
Epoch 4/200
157/157 [==============================] - 1s 8ms/step - loss: 0.5628 - accuracy: 0.7017 - val_loss: 0.7776 - val_accuracy: 0.6927 - lr: 0.0050
Epoch 5/200
157/157 [==============================] - 1s 7ms/step - loss: 0.5589 - accuracy: 0.7026 - val_loss: 0.8239 - val_accuracy: 0.6747 - lr: 0.0050
Epoch 6/200
157/157 [==============================] - 1s 7ms/step - loss: 0.5567 - accuracy: 0.7015 - val_loss: 0.8674 - val_accuracy: 0.6887 - lr: 0.0050
Epoch 7/200
157/157 [==============================] - 1s 7ms/st

Epoch 1/200
157/157 [==============================] - 2s 8ms/step - loss: 0.5977 - accuracy: 0.6811 - val_loss: 0.5859 - val_accuracy: 0.6856 - lr: 0.0050
Epoch 2/200
157/157 [==============================] - 2s 10ms/step - loss: 0.5671 - accuracy: 0.6985 - val_loss: 0.5705 - val_accuracy: 0.6953 - lr: 0.0050
Epoch 3/200
157/157 [==============================] - 2s 11ms/step - loss: 0.5598 - accuracy: 0.7006 - val_loss: 0.5666 - val_accuracy: 0.6896 - lr: 0.0050
Epoch 4/200
 96/157 [=================>............] - ETA: 0s - loss: 0.5560 - accuracy: 0.7015